In [0]:
import os

In [0]:
import pandas as pd

In [0]:
os.system("pip install -r https://raw.githubusercontent.com/George-Michael-Dagogo/World_news_tutorial/main/requirements.txt")

In [0]:
from newspaper import Article, Config

In [0]:
from nltk.corpus import stopwords

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import date, timedelta

In [0]:
from datetime import date, timedelta

In [0]:
from newsapi import NewsApiClient

In [0]:
from nltk.tokenize import word_tokenize

In [0]:
# Function to extract content from a URL
def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)

    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'

In [0]:
%python
def count_words_without_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound'], sentiment_scores['compound']

def full_content(url):
    # Dummy implementation for the full_content function
    return "This is a dummy content for the URL."

def extract_transform_function():
    today = date.today()                        
    yesterday = today - timedelta(days=1)     
    day_before_yesterday = today - timedelta(days=2)   
    
    # Initialize the News API client with an API key
    newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

    top_headlines = newsapi.get_top_headlines(category='entertainment',language='en',page_size=90,page=1)

    articles = top_headlines.get('articles', [])  # Extract articles from the API response

    # Creating DF from the articles, selecting specific columns
    init_df = pd.DataFrame(articles, columns=['source', 'title', 'publishedAt', 'author', 'url'])

    init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

    init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])
    filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | 
                          (init_df['publishedAt'].dt.date == yesterday)]
    filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
    
    df = filtered_df.copy()  # Copy of filtered DF

    df['content'] = df['url'].apply(full_content)  # Apply the full_content function to each URL in the DataFrame
    
    df['content'] = df['content'].str.replace('\n', ' ')  # Replace newlines in the 'content' column with spaces
    
    df = df[df['content'] != 'couldnt retrieve']  # Filter out rows where the content could not be retrieved

    # Download the NLTK stopwords dataset and other required datasets
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')

    # Apply the word count function to the 'content' column
    df['word_count'] = df['content'].apply(count_words_without_stopwords)

    
    nltk.download('vader_lexicon')

    
    sid = SentimentIntensityAnalyzer()    # Initialize the SentimentIntensityAnalyzer

    # Apply the sentiment analysis function to the 'content' column
    df[['sentiment', 'compound_score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))

    

    return df
        
dataframe = extract_transform_function()

In [0]:
# Function to count words excluding stopwords
def count_words_without_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

# Function for sentiment analysis
def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound'], sentiment_scores['compound']

def full_content(url):
    # Dummy implementation for the full_content function
    return "This is a dummy content for the URL."

# Data extraction and transformation
def extract_transform_function():
    today = date.today()                        
    yesterday = today - timedelta(days=1)     
    day_before_yesterday = today - timedelta(days=2)   
    
    # Initialize News API client
    newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

    top_headlines = newsapi.get_top_headlines(category='entertainment', language='en', page_size=90, page=1)

    articles = top_headlines.get('articles', [])  # Extract articles

    # Convert articles to DataFrame
    init_df = pd.DataFrame(articles, columns=['source', 'title', 'publishedAt', 'author', 'url'])

    # Extract source name safely
    init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

    # Convert 'publishedAt' to date format and filter
    init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])
    filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | 
                          (init_df['publishedAt'].dt.date == yesterday)]
    
    filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
    
    df = filtered_df.copy()  # Copy DataFrame
    df['content'] = df['url'].apply(full_content)  # Fetch article content
    
    # Clean text formatting
    df['content'] = df['content'].str.replace('\n', ' ')
    
    # Remove entries where content couldn't be retrieved
    df = df[df['content'] != 'couldnt retrieve']

    # Download required NLTK datasets
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('vader_lexicon')

    # Calculate word count
    df['word_count'] = df['content'].apply(count_words_without_stopwords)


    # Perform sentiment analysis
    df[['sentiment', 'compound_score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))


    return df

# Extract transformed data
dataframe = extract_transform_function()


In [0]:
dataframe.display()

In [0]:
print(dataframe.dtypes)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS the_news;
CREATE TABLE IF NOT EXISTS the_news.news_table (
source STRING,
title STRING,
date_posted DATE,
author STRING,
url STRING,
content STRING,
word_count INT,
sentiment STRING,
compound_score DOUBLE
)

In [0]:
%sql DESCRIBE FORMATTED the_news.news_table;


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, DoubleType

# Initialize Spark session
spark = SparkSession.builder.appName("CreateTableExample").getOrCreate()

# Convert sentiment to string for Spark compatibility
dataframe['sentiment'] = dataframe['sentiment'].fillna('').astype(str)

# Define schema explicitly
schema = StructType([
    StructField("source", StringType(), True),
    StructField("title", StringType(), True),
    StructField("date_posted", DateType(), True), 
    StructField("author", StringType(), True),
    StructField("url", StringType(), True),
    StructField("content", StringType(), True),
    StructField("word_count", IntegerType(), True),
    StructField("sentiment", StringType(), True),
    StructField("compound_score", DoubleType(), True)
])

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(dataframe, schema=schema)

# ***CHANGE HERE: Use 'append' instead of 'overwrite'***
spark_df.write.mode('append').saveAsTable('the_news.news_table')


In [0]:
%sql
select * from the_news.news_table